In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
''''
import random

with open('emails_spam_ham.csv', 'r') as r, open('shuffled_emails_spam_ham.csv', 'w') as w:
    data = r.readlines()
    header, rows = data[0], data[1:]
    random.shuffle(rows)
    rows = '\n'.join([row.strip() for row in rows])
    w.write(header + rows)
'''


"'\nimport random\n\nwith open('emails_spam_ham.csv', 'r') as r, open('shuffled_emails_spam_ham.csv', 'w') as w:\n    data = r.readlines()\n    header, rows = data[0], data[1:]\n    random.shuffle(rows)\n    rows = '\n'.join([row.strip() for row in rows])\n    w.write(header + rows)\n"

In [2]:
data = pd.read_csv('shuffled_emails_spam_ham.csv')

In [3]:
print(data.head())
data.columns
data.drop_duplicates()#removes same values
data.shape

                                                text  spam
0  Subject: organizational changes  to : enron no...     0
1  Subject: luigi zingales seminar on april 27  r...     0
2  Subject: it works greatt  hello , welcome to m...     1
3  Subject: spring 2001 academic calendar for the...     0
4  Subject: re : nick bambos  sounds great . let ...     0


(5728, 2)

In [4]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer


[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [5]:
data['text'] = data['text'].map(lambda text : text[8:]) #removes the word subject from all the emails 
#print(data['text'])
data['text'] = data['text'].map(lambda text: re.sub('[^A-Za-z0-9]+', ' ',text)) #removes all alphanumeric char
#print(data['text'])



In [6]:
data['text'] = data['text'].apply(lambda x: (x.lower()).split())
#print(data['text'])
#converts all to lower case and splits into indivisual words

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
ps = PorterStemmer()
corpus=data['text'].apply(lambda text_list:' '.join(list(map(lambda word:ps.stem(word),(list(filter(lambda text:text not in set(stopwords.words('english')),text_list)))))))

print(len(corpus))
#after_removing_stop_words = (filter(lambda text:text not in set(stopwords.words('english')),data['text']))
                         
#stemming = (map(lambda word:ps.stem(word), after_removing_stop_words))
                                 
#corpus = data['text'].apply(lambda text_list:' '.join(stemming))

                                 
#print(after_removing_stop_words)

5728


In [8]:
cv = CountVectorizer()
x = cv.fit_transform(corpus.values).toarray()#https://www.youtube.com/watch?v=RZYjsw6P4nI
print(x)#unique words are extracted and a bag of words thing is created , you can see the number of time seach word is reapeted by cheaking the to_array thing
 

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [10]:
y = data.iloc[:, 1].values #it sliced thhe array taking all the rows(:) and keeping the first columb(1)
print(y)

[0 0 1 ... 0 0 0]


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.20)
classifier = MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
classifier.fit(X_train , y_train)
y_pred = classifier.predict(X_test)


In [12]:
cm = confusion_matrix(y_test, y_pred)


In [13]:
print(accuracy_score(y_test, y_pred) )
accuracy_score(y_test, y_pred,normalize=False) 

0.9886561954624782


1133

In [ ]:
from sklearn.model_selection import learning_curve
train_sizes, train_scores, test_scores = learning_curve(classifier, x, y,cv=10, scoring='accuracy', n_jobs=-1, train_sizes=np.linspace(0.01, 1.0, 10))
